Кирилл Лалаянц, R33352

# Лабораторная работа No7
# В -- 11
## Управляемость и наблюдаемость

Импорт необходимых для работы библиотек. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os

SAVE_PATH = 'tex-report/figs/'
os.makedirs(SAVE_PATH, exist_ok=True)

sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")
w = sympy.Symbol("w")
I = sympy.I


def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))